In [1]:
import jax
import jax.numpy as jnp
import numpy as np

In [2]:
from arraytainers import Arraytainer, Jaxtainer

In [3]:
import copy
copy.copy([jnp.array(1)])

[DeviceArray(1, dtype=int32, weak_type=True)]

In [4]:
val = Arraytainer({'a': 'C', 'b': [{'a': 'F'}]}, array_conversions=False)

{'a': 'C', 'b': [{'a': 'F'}]}
False
b
[{'a': 'F'}]
False
b
{'a': 'F'}
False
{'a': 'F'}
False
{'a': 'F'}
True
c
[Arraytainer({'a': array('F', dtype='<U1')})]
False
a


In [5]:
val

Arraytainer({'a': 'C', 'b': [{'a': array('F', dtype='<U1')}]})

In [6]:
val._recursive_arraytainer_conversion(val, dtype=None, array_conversions=False)

Arraytainer({'a': 'C', 'b': [{'a': array('F', dtype='<U1')}]})
False
a


Arraytainer({'a': 'C', 'b': [{'a': array('F', dtype='<U1')}]})